In [99]:
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from numba import cuda

In [100]:
def mass_search_CPU(N, H, R):
  for j in range(R.shape[1]):
    for i in range(R.shape[0]):
      n = N[i]
      for k in range(len(n)):
        if n[k] == H[j]:
          R[i, j - k] -= 1
  return R

In [123]:
@cuda.jit
def mass_search_GPU(N, H, R):
	# Получение индекса потока в сетке
	x, y = cuda.grid(2)
	
	for k in range(N.shape[1]):
		# Если символ подстроки совпал с символом из буффера то уменьшаем значение элемента
    # в таблице (в строке таблицы = номер подстроки, столбце = позиция символа в буфере)
		if N[y, k] == H[x]:
				R[y, x - k] -= 1
	cuda.syncthreads()

In [102]:
def save_input(pathfile, N, H, mode='a', sheet_name=''):

  # Приведение списка символов (подстроки) к типу str
  # для корректной записи в excel
  new_N = [str(n) for n in N]
  
  # Создание DataFrame
  df = pd.DataFrame({"N": new_N, "H": H})
  
  # Если стоит mode='a', то первая запись в файл выдаст ошибку,
  # поэтому в этом случае mode меняется на 'w'.
  try:
    with pd.ExcelWriter(pathfile, mode=mode) as writer:
      df.to_excel(writer, sheet_name=sheet_name)
  except:
    with pd.ExcelWriter(pathfile, mode='w') as writer:
      df.to_excel(writer, sheet_name=sheet_name)

In [103]:
# Сохраняем пары (номер подстроки и позиция с которой она входит в буффер)
def find_N_i_and_k(R):
  result_list=[]
  for i in range(R.shape[0]):
    for j in range(R.shape[1]):
      if R[i,j]==0:
        result_list.append((i,j))
  return result_list

In [128]:
# Блок инициализации 
sizes = np.arange(160, 1600, 160)  # Размерности N и H
i = 5                             # Кол-во итераций для подсчёта среднего времени
N_len = 2                         # Кол-во символов в подстроках
BLOCK_DIM = 32                    # Размерность оси квадратного блока
BLOCK_SIZE = (BLOCK_DIM, BLOCK_DIM)

E = np.arange(256)

In [133]:
df = {"GPU time":np.zeros(len(sizes), dtype=float), 
      "CPU time":np.zeros(len(sizes), dtype=float),
      "Matching results":np.full(len(sizes), fill_value=True)}


df = pd.DataFrame(df, index=sizes)

print("Вывод массивов содержащих результаты поиска подстроки в буфере")
print("Первый элемент в парах - номер подстроки, которая вошла в буфер, второй элемент - позиция с которой идет вхождение")
# Вычисления для разных размерностей N и H
for size in sizes:

  # Случайная генерация подстрок N и входного буффера H
  N = np.random.randint(len(E), size=(size, N_len), dtype=np.uint8)
  H = np.random.randint(len(E), size=size, dtype=np.uint8)

  # Инициализация сетки блоков
  GRID_SIZE = ((len(N) + BLOCK_DIM - 1) // BLOCK_DIM , (len(H) + BLOCK_DIM - 1) // BLOCK_DIM)

  # Суммы общего времени за определённое кол-во итераций для реализации
  gpu_common_time = []
  cpu_common_time = []

  
  for _ in range (i + 1):

    # Заполнение рабочей матрицы NxH значениями N_len
    R = np.full((size, size), fill_value=N_len)
   
    
    # Пересылка данных на устройство
    dev_N = cuda.to_device(N)
    dev_H = cuda.to_device(H)
    dev_R = cuda.to_device(R)

    #Вычисление времени на GPU без учета пересылки данных
    start_time = time()

    # Запуск вычислений на GPU
    mass_search_GPU[GRID_SIZE, BLOCK_SIZE](dev_N, dev_H, dev_R)

    # Копирование рабочей матрицы с устройства на хост
    host_R = dev_R.copy_to_host()
    gpu_common_time.append(time() - start_time)

    #Вычисление времени на CPU
    start_time = time()
    R = mass_search_CPU(N, H, R.copy())
    cpu_common_time.append(time() - start_time)

    if _ == 0:
      
      print(f"Результаты для размера буфера - {size} ")
      print(find_N_i_and_k(host_R))

      save_input("input_data.xlsx", N, H, sheet_name=f'Размер {size}')


  # Запись подсчётов времени и соответствия рабочих матриц
  df.loc[size, "GPU time"] = np.mean(gpu_common_time[0:])
  df.loc[size, "CPU time"] = np.mean(cpu_common_time[0:])
  df.loc[size, "Matching results"] = np.array_equal(host_R, R)


# Запись значений ускорения GPU над CPU
df["Speed ​​up"] = df["CPU time"] / df["GPU time"]
df

Вывод массивов содержащих результаты поиска подстроки в буфере
Первый элемент в парах - номер подстроки, которая вошла в буфер, второй элемент - позиция с которой идет вхождение
Результаты для размера буфера - 160 
[]
Результаты для размера буфера - 320 
[(15, 264), (89, 13), (178, 126), (245, 11)]
Результаты для размера буфера - 480 
[(37, 102), (111, 218), (251, 332)]
Результаты для размера буфера - 640 
[(18, 291), (113, 135), (115, 282), (219, 577), (277, 170), (374, 82), (508, 619), (580, 463)]
Результаты для размера буфера - 800 
[(124, 296), (128, 343), (260, 713), (297, 684), (406, 432), (462, 698), (481, 731), (517, 340), (530, 789), (780, 604)]
Результаты для размера буфера - 960 
[(49, 710), (137, 438), (169, 309), (247, 194), (255, 240), (279, 439), (291, 219), (291, 698), (291, 883), (302, 360), (430, 138), (441, 559), (491, 33), (516, 647), (573, 199), (668, 477), (732, 442), (743, 81), (751, 194)]
Результаты для размера буфера - 1120 
[(7, 375), (27, 321), (52, 286), (21

,GPU time,CPU time,Matching results,Speed ​​up
160,0.000349,0.020250,True,58.088795
320,0.000676,0.082475,True,121.990714
480,0.001014,0.185546,True,183.021283
640,0.001356,0.333566,True,245.927023
800,0.001824,0.521132,True,285.767045
960,0.002385,0.735847,True,308.492837
1120,0.003139,1.018240,True,324.398142
1280,0.003758,1.306128,True,347.578420
1440,0.004584,1.653805,True,360.796973
